# Demo RNA Multi-Perceptrón Backpropagation para clasificar (atributo clase discreto) o estimar (atributo clase continuo)

In [1]:
#@title Librerías a usar
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import tensorflow as tf
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import math

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import os
import csv

import ipywidgets as widgets
from ipywidgets import Box, Layout
from IPython.display import clear_output
import random

from model.loader import CustomDataset
from model.model import Model
from model.trainer import train_model
from torch.utils.data import DataLoader
from analysis.logger import Logger
from model.LinearClamp import LinearClamp

import torch
import torch.optim as optim
import torch.nn as nn

print("Librerías cargadas")

2025-11-21 14:02:53.226500: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-21 14:02:53.414806: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-21 14:02:56.543603: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Librerías cargadas


## Pre-procesamiento de datos

In [2]:
!wget https://raw.githubusercontent.com/juanfra18/TP-IAA/main/datos/Resilience_CleanOnly_v1.csv -O Resilience_CleanOnly_v1.csv

--2025-11-21 14:03:00--  https://raw.githubusercontent.com/juanfra18/TP-IAA/main/datos/Resilience_CleanOnly_v1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1402236 (1,3M) [application/octet-stream]
Saving to: ‘Resilience_CleanOnly_v1.csv’

Resilience_CleanOnl 100%[===================>]   1,34M  --.-KB/s    in 0,1s    

2025-11-21 14:03:00 (12,6 MB/s) - ‘Resilience_CleanOnly_v1.csv’ saved [1402236/1402236]



In [3]:
original_data = pd.read_csv('datos/Resilience_CleanOnly_v1.csv', encoding='cp1252', delimiter=',')
original_data.head()

,ppt_internal_id,Dataset,r8.2_who5_num,r8.2_who5_fct4,r8.2_who5_Iexcel,age_fct6,gender_fct2,eth_fct4,education_qual,r4_previnworkforce_fct3,...,r8.16_2_fct2,r8.16_3_fct2,r8.16_6_fct2,r8.16_4_fct2,r12.1_11_fct2,r12.1_13_fct2,r6.4_fct2,r6.5_fct2,r6_change_fct3,postweight_unscaled
0,1,Dynata,13,Good,Other,15-24,Male,European/Other^,High school,Employed,...,No,No,No,No,No,Yes,Low level,Low level,Increase,3402.795918
1,2,Dynata,21,Very good,Other,55-64,Male,European/Other^,Certificate or diploma,Retired,...,No,No,No,No,Yes,No,Hazardous level,Hazardous level,No change,2250.293478
2,3,Dynata,16,Good,Other,25-34,Female,Pacific,Bachelor or above,Employed,...,No,No,Yes,No,No,No,Low level,Low level,No change,1659.857143
3,4,Dynata,4,Poor,Other,45-54,Female,European/Other^,Bachelor or above,Employed,...,Yes,No,No,Yes,Yes,No,Low level,Low level,Increase,2132.097087
4,5,Dynata,7,Poor,Other,15-24,Female,European/Other^,Certificate or diploma,Employed,...,Yes,Yes,Yes,Yes,Yes,Yes,Low level,Low level,Increase,2008.063291


In [4]:
original_data

,ppt_internal_id,Dataset,r8.2_who5_num,r8.2_who5_fct4,r8.2_who5_Iexcel,age_fct6,gender_fct2,eth_fct4,education_qual,r4_previnworkforce_fct3,...,r8.16_2_fct2,r8.16_3_fct2,r8.16_6_fct2,r8.16_4_fct2,r12.1_11_fct2,r12.1_13_fct2,r6.4_fct2,r6.5_fct2,r6_change_fct3,postweight_unscaled
0,1,Dynata,13,Good,Other,15-24,Male,European/Other^,High school,Employed,...,No,No,No,No,No,Yes,Low level,Low level,Increase,3402.795918
1,2,Dynata,21,Very good,Other,55-64,Male,European/Other^,Certificate or diploma,Retired,...,No,No,No,No,Yes,No,Hazardous level,Hazardous level,No change,2250.293478
2,3,Dynata,16,Good,Other,25-34,Female,Pacific,Bachelor or above,Employed,...,No,No,Yes,No,No,No,Low level,Low level,No change,1659.857143
3,4,Dynata,4,Poor,Other,45-54,Female,European/Other^,Bachelor or above,Employed,...,Yes,No,No,Yes,Yes,No,Low level,Low level,Increase,2132.097087
4,5,Dynata,7,Poor,Other,15-24,Female,European/Other^,Certificate or diploma,Employed,...,Yes,Yes,Yes,Yes,Yes,Yes,Low level,Low level,Increase,2008.063291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3482,3483,Ministries,20,Very good,Other,65+,Female,Pacific,No formal qualification,Employed,...,No,No,No,Yes,Yes,Yes,Low level,Low level,No change,2661.000000
3483,3484,Ministries,6,Poor,Other,45-54,Male,Maori,No formal qualification,Employed,...,No,No,No,No,No,No,Low level,Low level,No change,2315.666667
3484,3485,Ministries,7,Poor,Other,25-34,Male,European/Other^,Certificate or diploma,Employed,...,No,Yes,No,Yes,Yes,Yes,Low level,Low level,No change,4053.000000
3485,3486,Ministries,10,Poor,Other,45-54,Female,European/Other^,High school,Employed,...,No,No,No,No,Yes,No,Low level,Low level,No change,1557.489362


In [5]:
original_data[['r3.3_fct5','r3.3_num']]
#Prefiero quedarme con r3.3_fct5 ya que es más facil luego encontrar patrones entre los datos si están categorizados y no tienen valores sueltos. sin embargo, la segunda variable no tiene valores nulos mientras que la primera si, asi q la transformo luego

#Quito r3.3_num, r8.2_who5_fct4, r8.2_who5_Iexcel, r3.3_num
#postweight_unscaled no es propiedad real del caso, tal vez pueden usarse como pesos para el entrenamiento¿?

,r3.3_fct5,r3.3_num
0,6-9,6
1,2,2
2,3-5,3
3,3-5,4
4,1,1
...,...,...
3482,2,2
3483,3-5,5
3484,3-5,3
3485,3-5,3


In [6]:
original_data[['postweight_unscaled']]

,postweight_unscaled
0,3402.795918
1,2250.293478
2,1659.857143
3,2132.097087
4,2008.063291
...,...
3482,2661.000000
3483,2315.666667
3484,4053.000000
3485,1557.489362


In [7]:
rename_col = {
    'r8.2_who5_num': 'target',

    'age_fct6': 'age_group',
    'gender_fct2': 'gender',
    'eth_fct4': 'ethnicity',
    'r4_previnworkforce_fct3': 'employment_pre_lockdown',
    'r4.5_fct3': 'work_type',
    'r7.1_fct3': 'smoking_status',
    'r5.2_fct2': 'self_rated_health',
    'r8.17_fct2': 'mental_health_diagnosis',
    'r5.10': 'physical_disability',
    'r9.1_fct2': 'trauma_exposure',
    'r3.2_fct4': 'bubble_type',
  # 'r3.3_fct5': 'bubble_size_cat',
    'r3.3_num': 'bubble_size_num',
    'r3.7_fct3': 'social_connection_freq',
    'r3.8': 'contact_change',
    'r4_lesswork_fct2': 'less_work',
    'r4_lostwork_fct2': 'lost_work',
    'r5.6_fct3': 'covid_exposure',
    'r3.4': 'bubble_satisfaction',
    'r3.10': 'bubble_relationships',
    'r3.11': 'loneliness',
    'r3.12': 'time_covid_info',
    'r8.16_1_fct2': 'stress_self_health',
    'r8.16_2_fct2': 'stress_family_health',
    'r8.16_3_fct2': 'stress_finances',
    'r8.16_6_fct2': 'stress_employment',
    'r8.16_4_fct2': 'stress_covid_conseq',
    'r12.1_11_fct2': 'positive_lockdown_personal',
    'r12.1_13_fct2': 'positive_lockdown_society',
    'r6.4_fct2': 'alcohol_pre',
    'r6.5_fct2': 'alcohol_during',
    'r6_change_fct3': 'alcohol_change',
}

In [8]:
data = original_data.copy()

In [9]:
data = data.drop(columns=['r3.3_fct5', 'r8.2_who5_fct4', 'r8.2_who5_Iexcel','postweight_unscaled','ppt_internal_id','Dataset'])

In [10]:
data = data.rename(columns = rename_col)

In [11]:
data.shape

(3487, 34)

# Missing values

## Total missing values

In [12]:
(data.isnull().sum().sort_values(ascending=False)/len(data))*100

lost_work                     45.081732
loneliness                    42.299971
less_work                     39.116719
work_type                     39.088041
bubble_relationships          15.256668
mental_health_diagnosis        2.208202
social_connection_freq         2.122168
gender                         0.630915
contact_change                 0.573559
positive_lockdown_personal     0.458847
positive_lockdown_society      0.458847
income_band                    0.401491
alcohol_change                 0.229424
alcohol_during                 0.200746
alcohol_pre                    0.143390
bubble_type                    0.114712
time_covid_info                0.086034
smoking_status                 0.086034
employment_pre_lockdown        0.028678
covid_exposure                 0.028678
age_group                      0.000000
target                         0.000000
education_qual                 0.000000
ethnicity                      0.000000
physical_disability            0.000000


In [13]:
data.isnull().sum().sort_values(ascending=False)

lost_work                     1572
loneliness                    1475
less_work                     1364
work_type                     1363
bubble_relationships           532
mental_health_diagnosis         77
social_connection_freq          74
gender                          22
contact_change                  20
positive_lockdown_personal      16
positive_lockdown_society       16
income_band                     14
alcohol_change                   8
alcohol_during                   7
alcohol_pre                      5
bubble_type                      4
time_covid_info                  3
smoking_status                   3
employment_pre_lockdown          1
covid_exposure                   1
age_group                        0
target                           0
education_qual                   0
ethnicity                        0
physical_disability              0
self_rated_health                0
bubble_size_num                  0
trauma_exposure                  0
bubble_satisfaction 

## target

In [14]:
data['target'].value_counts(dropna=False)

target
20    337
19    274
17    236
18    223
15    219
16    207
14    198
13    182
21    150
12    143
10    142
9     132
11    128
5     125
22    102
8      98
6      97
7      92
25     85
4      70
3      62
23     62
24     54
2      24
.      19
1      14
0      12
Name: count, dtype: int64

In [15]:
# Mantener solo valores numéricos <= 25 y distintos de '.'
col = pd.to_numeric(
    data['target'].astype(str).str.strip().replace({'.': np.nan, '': np.nan}),
    errors='coerce'
)
data = data[col.le(25)].copy()  # descarta NaN y >25
data['target'] = col.loc[data.index].astype('Int64')
data['target'].value_counts(dropna=False)

target
20    337
19    274
17    236
18    223
15    219
16    207
14    198
13    182
21    150
12    143
10    142
9     132
11    128
5     125
22    102
8      98
6      97
7      92
25     85
4      70
3      62
23     62
24     54
2      24
1      14
0      12
Name: count, dtype: Int64

## employment_pre_lockdown

In [16]:
table = 'employment_pre_lockdown'
data[table].value_counts(dropna=False)

employment_pre_lockdown
Employed      2230
Retired        661
Unemployed     576
NaN              1
Name: count, dtype: int64

In [17]:
most_common = data[table].value_counts(dropna=False).idxmax()
data[table] = np.where(data[table].isnull(), most_common, data[table])
data[table].value_counts(dropna=False)

employment_pre_lockdown
Employed      2231
Retired        661
Unemployed     576
Name: count, dtype: int64

## bubble_type

In [18]:
table = 'bubble_type'
data[table].value_counts(dropna=False)

bubble_type
With children        1149
With one adult       1103
With other adults     685
Live by myself        527
NaN                     4
Name: count, dtype: int64

In [19]:
most_common = data[table].value_counts(dropna=False).idxmax()
data[table] = np.where(data[table].isnull(), most_common, data[table])
data[table].value_counts(dropna=False)

bubble_type
With children        1153
With one adult       1103
With other adults     685
Live by myself        527
Name: count, dtype: int64

## lost_work

In [20]:
data['lost_work'].value_counts(dropna=False)

lost_work
Not lost work    1797
NaN              1559
Lost work         112
Name: count, dtype: int64

In [21]:
(data['employment_pre_lockdown']!='Employed').sum()
#Quienes directamente no tenían trabajo previo al COVID no pueden perderlo

1237

In [22]:
data['lost_work'] = np.where(data['lost_work'].isnull() & (data['employment_pre_lockdown']!='Employed'), 'Without work', data['lost_work'])
data['lost_work'].value_counts()

lost_work
Not lost work    1797
Without work     1237
Lost work         112
Name: count, dtype: int64

In [23]:
nulos = data['lost_work'].isnull().sum()
print('Porcentaje de nulos: ', (nulos/len(data))*100)
print('Cantidad de nulos: ', nulos)

Porcentaje de nulos:  9.284890426758938
Cantidad de nulos:  322


In [24]:
data['lost_work'] = np.where(data['lost_work'].isnull(), 'Not lost work', data['lost_work'])

## loneliness

In [25]:
data[['loneliness']].value_counts()
#Los nulos en realidad corresponden a la categoría 'none'

loneliness        
More than a little    1003
A little               996
Name: count, dtype: int64

In [26]:
data['loneliness'] = np.where(data['loneliness'].isnull(), 'None', data['loneliness'])
data[['loneliness']].value_counts()

loneliness        
None                  1469
More than a little    1003
A little               996
Name: count, dtype: int64

## less_work

In [27]:
#Caso similar a lost_work

In [28]:
data['less_work'].value_counts()

less_work
Not less work    1348
Less work         769
Name: count, dtype: int64

In [29]:
data['less_work'] = np.where(data['less_work'].isnull() & (data['employment_pre_lockdown']!='Employed'), 'Without work', data['less_work'])
data['less_work'].value_counts()

less_work
Not less work    1348
Without work     1237
Less work         769
Name: count, dtype: int64

In [30]:
nulos = data['less_work'].isnull().sum()
print('Porcentaje de nulos: ', (nulos/len(data))*100)
print('Cantidad de nulos: ', nulos)

Porcentaje de nulos:  3.2871972318339098
Cantidad de nulos:  114


In [31]:
data['less_work'] = np.where(data['less_work'].isnull(), 'Not less work', data['less_work'])

## work_type


In [32]:
data['work_type'].value_counts(dropna=False)

work_type
Not essential worker    1395
NaN                     1350
Yes essential worker     723
Name: count, dtype: int64

In [33]:
data['work_type'] = np.where(data['work_type'].isnull() & (data['employment_pre_lockdown']!='Employed'), 'Without work', data['work_type'])
data['work_type'].value_counts(dropna=False)

work_type
Not essential worker    1395
Without work            1237
Yes essential worker     723
NaN                      113
Name: count, dtype: int64

In [34]:
nulos = data['work_type'].isnull().sum()
print('Porcentaje de nulos: ', (nulos/len(data))*100)
print('Cantidad de nulos: ', nulos)

Porcentaje de nulos:  3.2583621683967703
Cantidad de nulos:  113


In [35]:
data['work_type'] = np.where(data['work_type'].isnull(), 'Not essential worker', data['work_type'])

In [36]:
data['work_type'].value_counts(dropna=False)

work_type
Not essential worker    1508
Without work            1237
Yes essential worker     723
Name: count, dtype: int64

## bubble_size_num
Hago que sea 0 cuando no tiene bubble

In [37]:
data['bubble_size_num'].value_counts(dropna=False)

bubble_size_num
2      1176
3       617
4       574
1       527
5       320
6       118
7        56
8        26
9        15
.         9
10        8
12        5
13        4
11        4
25        1
14        1
19        1
40        1
21        1
45        1
999       1
15        1
70        1
Name: count, dtype: int64

In [38]:
# Mantener solo valores numéricos <= 50 y distintos de '.'
col = pd.to_numeric(
    data['bubble_size_num'].astype(str).str.strip().replace({'.': np.nan, '': np.nan}),
    errors='coerce'
)
data = data[col.le(50)].copy()  # descarta NaN y >50
data['bubble_size_num'] = col.loc[data.index].astype('Int64')
data['bubble_size_num'].value_counts(dropna=False)

bubble_size_num
2     1176
3      617
4      574
1      527
5      320
6      118
7       56
8       26
9       15
10       8
12       5
13       4
11       4
25       1
14       1
19       1
40       1
21       1
45       1
15       1
Name: count, dtype: Int64

## bubble_relationships

In [39]:
data['bubble_relationships'].value_counts(dropna=False)

bubble_relationships
Very well    1335
Well         1114
NaN           528
Not well      480
Name: count, dtype: int64

In [40]:
# Aquellos que viven solos no van a tener asociado bubble_relationships
len(data[data['bubble_relationships'].isnull() & (data['bubble_type']=='Live by myself')])

527

In [41]:
data['bubble_relationships'] = np.where(data['bubble_relationships'].isnull() & (data['bubble_type']=='Live by myself'), 'Without bubble', data['bubble_relationships'])
data['bubble_relationships'].value_counts(dropna=False)

bubble_relationships
Very well         1335
Well              1114
Without bubble     527
Not well           480
NaN                  1
Name: count, dtype: int64

In [42]:
data['bubble_relationships'] = np.where(data['bubble_relationships'].isnull(), 'Very well', data['bubble_relationships'])
data['bubble_relationships'].value_counts(dropna=False)

bubble_relationships
Very well         1336
Well              1114
Without bubble     527
Not well           480
Name: count, dtype: int64

## mental_health_diagnosis


In [43]:
data['mental_health_diagnosis'].value_counts(dropna=False)

mental_health_diagnosis
No                    2755
Yes past diagnosis     627
NaN                     75
Name: count, dtype: int64

In [44]:
nulos = data['mental_health_diagnosis'].isnull().sum()
print('Porcentaje de nulos: ', (nulos/len(data))*100)
print('Cantidad de nulos: ', nulos)

Porcentaje de nulos:  2.169511136823836
Cantidad de nulos:  75


In [45]:
data['mental_health_diagnosis'] = np.where(data['mental_health_diagnosis'].isnull(), 'No', data['mental_health_diagnosis'])
data['mental_health_diagnosis'].value_counts(dropna=False)

mental_health_diagnosis
No                    2830
Yes past diagnosis     627
Name: count, dtype: int64

## social_connection_freq


In [46]:
data['social_connection_freq'].value_counts(dropna=False)

social_connection_freq
High      1995
Medium     944
Low        447
NaN         71
Name: count, dtype: int64

In [47]:
nulos = data['social_connection_freq'].isnull().sum()
print('Porcentaje de nulos: ', (nulos/len(data))*100)
print('Cantidad de nulos: ', nulos)

Porcentaje de nulos:  2.053803876193231
Cantidad de nulos:  71


In [48]:
data['social_connection_freq'] = np.where(data['social_connection_freq'].isnull(), 'High', data['social_connection_freq'])
data['social_connection_freq'].value_counts(dropna=False)

social_connection_freq
High      2066
Medium     944
Low        447
Name: count, dtype: int64

## gender


In [49]:
data['gender'].value_counts(dropna=False)

gender
Female    1961
Male      1474
NaN         22
Name: count, dtype: int64

In [50]:
data['gender'] = np.where(data['gender'].isnull(), 'Female', data['gender'])
data['gender'].value_counts(dropna=False)

gender
Female    1983
Male      1474
Name: count, dtype: int64

## contact_change


In [51]:
data['contact_change'].value_counts(dropna=False)

contact_change
It has stayed the same    1231
It has increased          1117
It has decreased          1091
NaN                         18
Name: count, dtype: int64

In [52]:
data['contact_change'] = np.where(data['contact_change'].isnull(), 'It has stayed the same', data['contact_change'])
data['contact_change'].value_counts(dropna=False)

contact_change
It has stayed the same    1249
It has increased          1117
It has decreased          1091
Name: count, dtype: int64

## positive_lockdown_personal


In [53]:
data['positive_lockdown_personal'].value_counts(dropna=False)

positive_lockdown_personal
No     1741
Yes    1700
NaN      16
Name: count, dtype: int64

In [54]:
data['positive_lockdown_personal'] = np.where(data['positive_lockdown_personal'].isnull(), 'No', data['positive_lockdown_personal'])
data['positive_lockdown_personal'].value_counts(dropna=False)

positive_lockdown_personal
No     1757
Yes    1700
Name: count, dtype: int64

## positive_lockdown_society


In [55]:
data['positive_lockdown_society'].value_counts(dropna=False)

positive_lockdown_society
No     2003
Yes    1438
NaN      16
Name: count, dtype: int64

In [56]:
data['positive_lockdown_society'] = np.where(data['positive_lockdown_society'].isnull(), 'No', data['positive_lockdown_society'])
data['positive_lockdown_society'].value_counts(dropna=False)

positive_lockdown_society
No     2019
Yes    1438
Name: count, dtype: int64

## income_band


In [57]:
data['income_band'].value_counts(dropna=False)

income_band
$30,001– $70,000       1173
$30,000 or less        1114
$70,001 – $100,000      432
Prefer not to say       360
$100,001 – $150,000     238
$150,001 or more        126
NaN                      14
Name: count, dtype: int64

In [58]:
most_common = data['income_band'].value_counts(dropna=False).idxmax()
data['income_band'] = np.where(data['income_band'].isnull(), most_common, data['income_band'])
data['income_band'].value_counts(dropna=False)

income_band
$30,001– $70,000       1187
$30,000 or less        1114
$70,001 – $100,000      432
Prefer not to say       360
$100,001 – $150,000     238
$150,001 or more        126
Name: count, dtype: int64

## alcohol_change


In [59]:
data['alcohol_change'].value_counts(dropna=False)

alcohol_change
No change    1976
Increase      870
Decrease      603
NaN             8
Name: count, dtype: int64

In [60]:
data['alcohol_change'] = np.where(data['alcohol_change'].isnull(), 'No change', data['alcohol_change'])
data['alcohol_change'].value_counts(dropna=False)

alcohol_change
No change    1984
Increase      870
Decrease      603
Name: count, dtype: int64

## alcohol_during

In [61]:
data['alcohol_during'].value_counts(dropna=False)

alcohol_during
Low level          3017
Hazardous level     433
NaN                   7
Name: count, dtype: int64

In [62]:
data['alcohol_during'] = np.where(data['alcohol_during'].isnull(), 'Low level', data['alcohol_during'])
data['alcohol_during'].value_counts(dropna=False)

alcohol_during
Low level          3024
Hazardous level     433
Name: count, dtype: int64

## alcohol_pre

In [63]:
table = 'alcohol_pre'
data[table].value_counts(dropna=False)

alcohol_pre
Low level          3103
Hazardous level     349
NaN                   5
Name: count, dtype: int64

In [64]:
most_common = data[table].value_counts(dropna=False).idxmax()
data[table] = np.where(data[table].isnull(), most_common, data[table])
data[table].value_counts(dropna=False)

alcohol_pre
Low level          3108
Hazardous level     349
Name: count, dtype: int64

## time_covid_info

In [65]:
table = 'time_covid_info'
data[table].value_counts(dropna=False)

time_covid_info
Less than 2 hours    2975
Two plus hours        479
NaN                     3
Name: count, dtype: int64

In [66]:
most_common = data[table].value_counts(dropna=False).idxmax()
data[table] = np.where(data[table].isnull(), most_common, data[table])
data[table].value_counts(dropna=False)

time_covid_info
Less than 2 hours    2978
Two plus hours        479
Name: count, dtype: int64

## smoking_status

In [67]:
table = 'smoking_status'
data[table].value_counts(dropna=False)

smoking_status
Never      1868
Past       1043
Current     543
NaN           3
Name: count, dtype: int64

In [68]:
most_common = data[table].value_counts(dropna=False).idxmax()
data[table] = np.where(data[table].isnull(), most_common, data[table])
data[table].value_counts(dropna=False)

smoking_status
Never      1871
Past       1043
Current     543
Name: count, dtype: int64

## covid_exposure

In [69]:
table = 'covid_exposure'
data[table].value_counts(dropna=False)

covid_exposure
No           3299
Tested        119
Suspected      28
Confirmed      10
NaN             1
Name: count, dtype: int64

In [70]:
most_common = data[table].value_counts(dropna=False).idxmax()
data[table] = np.where(data[table].isnull(), most_common, data[table])
data[table].value_counts(dropna=False)

covid_exposure
No           3300
Tested        119
Suspected      28
Confirmed      10
Name: count, dtype: int64

In [71]:
data.isnull().sum().sort_values(ascending=False)

target                        0
age_group                     0
gender                        0
ethnicity                     0
education_qual                0
employment_pre_lockdown       0
work_type                     0
smoking_status                0
income_band                   0
self_rated_health             0
mental_health_diagnosis       0
physical_disability           0
trauma_exposure               0
bubble_type                   0
bubble_size_num               0
social_connection_freq        0
contact_change                0
less_work                     0
lost_work                     0
covid_exposure                0
bubble_satisfaction           0
bubble_relationships          0
loneliness                    0
time_covid_info               0
stress_self_health            0
stress_family_health          0
stress_finances               0
stress_employment             0
stress_covid_conseq           0
positive_lockdown_personal    0
positive_lockdown_society     0
alcohol_

In [72]:
data

,target,age_group,gender,ethnicity,education_qual,employment_pre_lockdown,work_type,smoking_status,income_band,self_rated_health,...,stress_self_health,stress_family_health,stress_finances,stress_employment,stress_covid_conseq,positive_lockdown_personal,positive_lockdown_society,alcohol_pre,alcohol_during,alcohol_change
0,13,15-24,Male,European/Other^,High school,Employed,Yes essential worker,Never,"$30,000 or less",Good or better,...,No,No,No,No,No,No,Yes,Low level,Low level,Increase
1,21,55-64,Male,European/Other^,Certificate or diploma,Retired,Without work,Never,"$70,001 – $100,000",Good or better,...,Yes,No,No,No,No,Yes,No,Hazardous level,Hazardous level,No change
2,16,25-34,Female,Pacific,Bachelor or above,Employed,Yes essential worker,Current,"$30,001– $70,000",Poor or Fair,...,No,No,No,Yes,No,No,No,Low level,Low level,No change
3,4,45-54,Female,European/Other^,Bachelor or above,Employed,Yes essential worker,Never,"$30,000 or less",Good or better,...,No,Yes,No,No,Yes,Yes,No,Low level,Low level,Increase
4,7,15-24,Female,European/Other^,Certificate or diploma,Employed,Not essential worker,Past,"$30,001– $70,000",Good or better,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Low level,Low level,Increase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3482,20,65+,Female,Pacific,No formal qualification,Employed,Not essential worker,Past,"$100,001 – $150,000",Good or better,...,No,No,No,No,Yes,Yes,Yes,Low level,Low level,No change
3483,6,45-54,Male,Maori,No formal qualification,Employed,Yes essential worker,Past,"$70,001 – $100,000",Good or better,...,No,No,No,No,No,No,No,Low level,Low level,No change
3484,7,25-34,Male,European/Other^,Certificate or diploma,Employed,Not essential worker,Never,"$30,001– $70,000",Good or better,...,No,No,Yes,No,Yes,Yes,Yes,Low level,Low level,No change
3485,10,45-54,Female,European/Other^,High school,Employed,Not essential worker,Never,"$70,001 – $100,000",Good or better,...,No,No,No,No,No,Yes,No,Low level,Low level,No change


# Normalización de datos:

In [73]:
data.head()

,target,age_group,gender,ethnicity,education_qual,employment_pre_lockdown,work_type,smoking_status,income_band,self_rated_health,...,stress_self_health,stress_family_health,stress_finances,stress_employment,stress_covid_conseq,positive_lockdown_personal,positive_lockdown_society,alcohol_pre,alcohol_during,alcohol_change
0,13,15-24,Male,European/Other^,High school,Employed,Yes essential worker,Never,"$30,000 or less",Good or better,...,No,No,No,No,No,No,Yes,Low level,Low level,Increase
1,21,55-64,Male,European/Other^,Certificate or diploma,Retired,Without work,Never,"$70,001 – $100,000",Good or better,...,Yes,No,No,No,No,Yes,No,Hazardous level,Hazardous level,No change
2,16,25-34,Female,Pacific,Bachelor or above,Employed,Yes essential worker,Current,"$30,001– $70,000",Poor or Fair,...,No,No,No,Yes,No,No,No,Low level,Low level,No change
3,4,45-54,Female,European/Other^,Bachelor or above,Employed,Yes essential worker,Never,"$30,000 or less",Good or better,...,No,Yes,No,No,Yes,Yes,No,Low level,Low level,Increase
4,7,15-24,Female,European/Other^,Certificate or diploma,Employed,Not essential worker,Past,"$30,001– $70,000",Good or better,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Low level,Low level,Increase


In [74]:
# Primero paso las columnas categóricas a números
mapping = {
        "age_group": {
            "15-24": 0,
            "25-34": 1,
            "35-44": 2,
            "45-54": 3,
            "55-64": 4,
            "65+": 5
        },
        "gender": {
            "Male": 0,
            "Female": 1
        },
        "ethnicity": {
            "European/Other^": 0,
            "Maori": 1,
            "Pacific": 2,
            "Asian": 3
        },
        "education_qual": {
            "No formal qualification": 0,
            "High school": 1,
            "Certificate or diploma": 2,
            "Bachelor or above": 3
        },
        "employment_pre_lockdown": {
            "Employed": 0,
            "Unemployed": 1,
            "Retired": 2
        },
        "work_type": {
            "Not essential worker": 0,
            "Yes essential worker": 1,
            "Without work": 2
        },
        "smoking_status": {
            "Never": 0,
            "Past": 1,
            "Current": 2
        },
        "income_band": {
            "$30,000 or less": 0,
            "$30,001– $70,000": 1,
            "$70,001 – $100,000": 2,
            "$100,001 – $150,000": 3,
            "$150,001 or more": 4,
            "Prefer not to say": 5
        },
        "self_rated_health": {
            "Poor or Fair": 0,
            "Good or better": 1
        },
        "mental_health_diagnosis": {
            "No": 0,
            "Yes past diagnosis": 1
        },
        "physical_disability": {
            "No": 0,
            "Yes": 1
        },
        "trauma_exposure": {
            "No": 0,
            "Yes": 1
        },
        "bubble_type": {
            "Live by myself": 0,
            "With one adult": 1,
            "With other adults": 2,
            "With children": 3
        },
        "social_connection_freq": {
            "High": 2,
            "Medium": 1,
            "Low": 0
        },
        "contact_change": {
            "It has stayed the same": 0,
            "It has increased": 1,
            "It has decreased": 2
        },
        "less_work": {
            "Not less work": 0,
            "Less work": 1,
            "Without work": 2
        },
        "lost_work": {
            "Not lost work": 0,
            "Lost work": 1,
            "Without work": 2
        },
        "covid_exposure": {
            "No": 0,
            "Suspected": 1,
            "Tested": 2,
            "Confirmed": 3
        },
        "bubble_satisfaction": {
            "Not satifisfied": 0,
            "Satisfied": 1,
            "Extremely satisfied": 2
        },
        "bubble_relationships": {
            "Not well": 0,
            "Well": 1,
            "Very well": 2,
            "Without bubble": 3
        },
        "loneliness": {
            "None": 0,
            "A little": 1,
            "More than a little": 2
        },
        "time_covid_info": {
            "Less than 2 hours": 0,
            "Two plus hours": 1
        },
        "stress_self_health": {
            "No": 0,
            "Yes": 1
        },
        "stress_family_health": {
            "No": 0,
            "Yes": 1
        },
        "stress_finances": {
            "No": 0,
            "Yes": 1
        },
        "stress_employment": {
            "No": 0,
            "Yes": 1
        },
        "stress_covid_conseq": {
            "No": 0,
            "Yes": 1
        },
        "positive_lockdown_personal": {
            "No": 0,
            "Yes": 1
        },
        "positive_lockdown_society": {
            "No": 0,
            "Yes": 1
        },
        "alcohol_pre": {
            "Low level": 0,
            "Hazardous level": 1
        },
        "alcohol_during": {
            "Low level": 0,
            "Hazardous level": 1
        },
        "alcohol_change": {
            "No change": 2,
            "Increase": 1,
            "Decrease": 0
        }
}

In [75]:
data_encoded = data.copy()

# 1) Reemplaza
data_encoded = data_encoded.replace(mapping)

# 2) Fuerza tipo numérico en las columnas mapeadas
for col, m in mapping.items():
    if col not in data_encoded.columns:
        continue
    if isinstance(m, dict) and len(m) == 0:
        # casos numéricos no codificados
        data_encoded[col] = pd.to_numeric(data_encoded[col], errors="coerce")
    elif isinstance(m, dict) and len(m) > 0:
        # columnas categóricas codificadas -> a enteros con soporte de NaN
        data_encoded[col] = pd.to_numeric(data_encoded[col], errors="coerce").astype("Int64")

# 3) (Opcional) chequeo de valores no mapeados
unmapped_report = {}
for col, m in mapping.items():
    if col in data.columns and isinstance(m, dict) and len(m) > 0:
        mask_unmapped = data[col].notna() & ~data[col].isin(m.keys())
        if mask_unmapped.any():
            unmapped_report[col] = sorted(data.loc[mask_unmapped, col].unique().tolist())

if unmapped_report:
    print("Categorías no mapeadas detectadas:", unmapped_report)

/tmp/ipykernel_561037/940831558.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_encoded = data_encoded.replace(mapping)


In [76]:
data_encoded

,target,age_group,gender,ethnicity,education_qual,employment_pre_lockdown,work_type,smoking_status,income_band,self_rated_health,...,stress_self_health,stress_family_health,stress_finances,stress_employment,stress_covid_conseq,positive_lockdown_personal,positive_lockdown_society,alcohol_pre,alcohol_during,alcohol_change
0,13,0,0,0,1,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,1
1,21,4,0,0,2,2,2,0,2,1,...,1,0,0,0,0,1,0,1,1,2
2,16,1,1,2,3,0,1,2,1,0,...,0,0,0,1,0,0,0,0,0,2
3,4,3,1,0,3,0,1,0,0,1,...,0,1,0,0,1,1,0,0,0,1
4,7,0,1,0,2,0,0,1,1,1,...,1,1,1,1,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3482,20,5,1,2,0,0,0,1,3,1,...,0,0,0,0,1,1,1,0,0,2
3483,6,3,0,1,0,0,1,1,2,1,...,0,0,0,0,0,0,0,0,0,2
3484,7,1,0,0,2,0,0,0,1,1,...,0,0,1,0,1,1,1,0,0,2
3485,10,3,1,0,1,0,0,0,2,1,...,0,0,0,0,0,1,0,0,0,2


In [77]:
data_encoded.to_csv('datos/Resilience_CleanOnly_v1_PREPROCESSED_v2.csv', index=False)

# Definir y entrenar el modelo

In [78]:
#train_data = CustomDataset(train_df, feat_cols, 'target', scaler_X, scaler_y)
#val_data   = CustomDataset(val_df,   feat_cols, 'target', scaler_X, scaler_y)

#train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
#val_loader   = DataLoader(val_data, batch_size=256, shuffle=False)

In [ ]:
df = pd.read_csv("datos/Resilience_CleanOnly_v1_PREPROCESSED_v2.csv", encoding="latin1")
sizes = [len(df.columns)-1,32,16,8,1]
output_activation = nn.Identity
intermediate_activation = nn.ReLU
normalize_output = True
loss = nn.MSELoss
name = f"model_{sizes}_output_{output_activation.__name__}_intermediate_{intermediate_activation.__name__}_normalized_data_{normalize_output}_loss_{loss.__name__}"

comparison_table = pd.read_csv("results/comparison_table.csv")

#probabilities = df["weight"]


weight_path = f"results/{name}.pth" if os.path.exists(f"results/{name}.pth") else None
logger = Logger("results/logs")

train_df = df.sample(frac=0.7, random_state=42)
val_df = df.drop(train_df.index)

train_data = CustomDataset(train_df, normalize_output)
val_data = CustomDataset(val_df, normalize_output)
train_loader = DataLoader(train_data, batch_size=2, shuffle=True)
val_loader = DataLoader(val_data, batch_size=2, shuffle=False)

model = Model(weight_path,description=name, hidden_sizes=sizes, output_activation=output_activation, dropout_p=0.5)  
criterion = loss()
optimizer = optim.Adagrad(model.parameters(), lr=0.1)

trained_model, val_loss = train_model(model, train_loader, val_loader, criterion, optimizer, 50, logger)

torch.save(trained_model.state_dict(), f"results/{name}.pth")

new_row = {
    "name": name,
    "loss": val_loss
}

comparison_table = pd.concat([comparison_table, pd.DataFrame([new_row])], ignore_index=True)
comparison_table.to_csv("results/comparison_table.csv", index=False)

[[2025-11-21 14:20:59]] Starting training on model: model_[33, 32, 16, 8, 1]_output_Identity_intermediate_ReLU_normalized_data_True_loss_MSELoss


/home/juan/Apps/miniconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[[2025-11-21 14:21:03]] Epoch 1/50, Train Loss: 0.05597545, Val Loss: 0.04867304


/home/juan/Apps/miniconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[[2025-11-21 14:21:07]] Epoch 2/50, Train Loss: 0.04903114, Val Loss: 0.04398182


...

[[2025-11-21 14:23:26]] Epoch 35/50, Train Loss: 0.04185612, Val Loss: 0.03621657


/home/juan/Apps/miniconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[[2025-11-21 14:23:31]] Epoch 36/50, Train Loss: 0.03949658, Val Loss: 0.03628550


/home/juan/Apps/miniconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[[2025-11-21 14:23:37]] Epoch 37/50, Train Loss: 0.04004390, Val Loss: 0.03694422
